In [34]:
import acquire
import pandas as pd
import numpy as np

In [35]:
# acquire the iris data and assign it into a variable called iris
iris = acquire.get_iris_data()

In [36]:
# check out iris to see if it exists here:
iris.head(2)

,species_id,species_name,sepal_length,sepal_width,petal_length,petal_width
0,1,setosa,5.1,3.5,1.4,0.2
1,1,setosa,4.9,3.0,1.4,0.2


In [37]:
# lets grab an arbitrary second iris from pydataset
# in order to demonstrate the column cleaning described below
# (we dont actually need this, just want to show how we would fix it)

from pydataset import data
iris_2 = data('iris')

In [38]:
# .columns refers to mutable property of the df:
# which means, we can actively reassign that value
iris_2.columns

Index(['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width',
       'Species'],
      dtype='object')

In [6]:
# if I want to change those values to being ones that happen
# to have underscores instead of periods,
# lets just change the ones we got, and reassign them
# .columns will always expect an iterable of the same width/lenth
# as it presently has, as it does not want to leave null columns or
# deal with more values than it has room for.
# initialize an empty list for the new column names
new_columns = []
# iterate through the old ones and "fix" them
for element in iris_2.columns:
    new_columns.append(element.replace('.', '_').lower())

In [7]:
# iris_2.columns = new_columns

In [8]:
iris_2.columns = [element.replace('.','_').lower() for element in iris_2.columns]

In [ ]:
# Drop the species_id and measurement_id columns.

In [9]:
iris.columns

Index(['species_id', 'species_name', 'sepal_length', 'sepal_width',
       'petal_length', 'petal_width'],
      dtype='object')

In [10]:
iris[['species_id', 'species_name']].head()

,species_id,species_name
0,1,setosa
1,1,setosa
2,1,setosa
3,1,setosa
4,1,setosa


In [11]:
pd.crosstab(iris.species_id, iris.species_name)

species_name,setosa,versicolor,virginica
species_id,,,
1,50,0,0
2,0,50,0
3,0,0,50


In [12]:
# drop that species_id column:
iris = iris.drop(columns='species_id')

In [13]:
iris.columns

Index(['species_name', 'sepal_length', 'sepal_width', 'petal_length',
       'petal_width'],
      dtype='object')

Rename the species_name column to just species.

In [14]:
# rename that species_name column into species for cleanliness:
iris = iris.rename(columns={'species_name':'species'})

Create a function named prep_iris that accepts the untransformed iris data, and returns the data with the transformations above applied.

In [16]:
# whip up a function to do the cleaning process on iris here:
def prep_iris(iris) -> pd.DataFrame:
    '''
    prep_iris will take a single positional argument,
    a single pandas DataFrame,
    and will output a cleaned version of the dataframe
    this is expected to receive the data output by 
    get_iris_data from acquire module, see documentation
    for acquire.py for further details
    return: pd.DataFrame
    '''
    # drop that species_id column:
    iris = iris.drop(columns='species_id')
    # rename that species_name column into species for cleanliness:
    iris = iris.rename(columns={'species_name':'species'})
    return iris

In [17]:
new_iris = acquire.get_iris_data()
new_iris.head(1)

,species_id,species_name,sepal_length,sepal_width,petal_length,petal_width
0,1,setosa,5.1,3.5,1.4,0.2


In [18]:
# lets check our pipeline by nesting the returns of our functions
clean_iris = prep_iris(acquire.get_iris_data())
clean_iris.head(2)

,species,sepal_length,sepal_width,petal_length,petal_width
0,setosa,5.1,3.5,1.4,0.2
1,setosa,4.9,3.0,1.4,0.2


Using the Titanic dataset

Use the function defined in acquire.py to load the Titanic data.

In [40]:
titanic_query = "SELECT * FROM passengers"

In [41]:
titanic = acquire.get_titanic_data(titanic_query, './')

In [42]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    891 non-null    int64  
 1   passenger_id  891 non-null    int64  
 2   survived      891 non-null    int64  
 3   pclass        891 non-null    int64  
 4   sex           891 non-null    object 
 5   age           714 non-null    float64
 6   sibsp         891 non-null    int64  
 7   parch         891 non-null    int64  
 8   fare          891 non-null    float64
 9   embarked      889 non-null    object 
 10  class         891 non-null    object 
 11  deck          203 non-null    object 
 12  embark_town   889 non-null    object 
 13  alone         891 non-null    int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 97.6+ KB


In [43]:
# lets get the sum of missing values by column:
titanic.isna().sum(axis=0)[titanic.isna().sum(axis=0) > 0]

age            177
embarked         2
deck           688
embark_town      2
dtype: int64

In [44]:
# percentage of deck missing is approximately 77
round((titanic.deck.isna().sum() / titanic.shape[0]) * 100)

77

In [45]:
# percent of age missing:
round((titanic.age.isna().sum() / titanic.shape[0]) * 100)

20

In [46]:
# how may we impute an age?:
# imputation refers to filling missing values here:
round(titanic.age.mean())

30

In [47]:
titanic.loc[:,'age'] = titanic.age.fillna(round(titanic.age.mean())).values

In [48]:
titanic = titanic.drop(columns='deck')

In [49]:
titanic[['embarked', 'embark_town']].head()

,embarked,embark_town
0,S,Southampton
1,C,Cherbourg
2,S,Southampton
3,S,Southampton
4,S,Southampton


In [50]:
pd.crosstab(titanic.embarked, titanic.embark_town)

embark_town,Cherbourg,Queenstown,Southampton
embarked,,,
C,168,0,0
Q,0,77,0
S,0,0,644


In [83]:
# lets impute the missing values for embark_town
# with the results of the most popular town (Southampton)
titanic = titanic.drop(columns= 'embarked')
titanic.loc[:, 'embark_town'] = titanic.embark_town.fillna('Southampton')

KeyError: "['embarked'] not found in axis"

titanic.head()

Create a function named prep_titanic that accepts the raw titanic data, and returns the data with the transformations above applied.

In [53]:
data('titanic', show_doc=True)

titanic

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## titanic

### Description

The data is an observation-based version of the 1912 Titanic passenger
survival log,

### Usage

    data(titanic)

### Format

A data frame with 1316 observations on the following 4 variables.

`class`

a factor with levels `1st class` `2nd class` `3rd class` `crew`

`age`

a factor with levels `child` `adults`

`sex`

a factor with levels `women` `man`

`survived`

a factor with levels `no` `yes`

### Details

titanic is saved as a data frame. Used to assess risk ratios

### Source

Found in many other texts

### References

Hilbe, Joseph M (2014), Modeling Count Data, Cambridge University Press Hilbe,
Joseph M (2007, 2011), Negative Binomial Regression, Cambridge University
Press Hilbe, Joseph M (2009), Logistic Regression Models, Chapman & Hall/CRC

### Examples

    data(titanic)
    titanic$survival <- titanic$survived == "yes"
    glmlr <- glm(survival ~

In [71]:
def prep_titanic(titanic) -> pd.DataFrame:
    '''
    prep_titanic will take in a single pandas DataFrame, titanic
    as expected from the acquire.py return of get_titanic_data
    it will return a single cleaned pandas dataframe
    of this titanic data, ready for analysis.
    '''
    titanic = titanic.drop(columns=[
        'Unnamed: 0',
        'passenger_id',
        'embarked',
        'deck',
        'class'
    ])
    titanic.loc[:,'age'] = titanic.age.fillna(round(titanic.age.mean())).values
    titanic.loc[:, 'embark_town'] = titanic.embark_town.fillna('Southampton')
    return titanic

In [72]:
titanic = prep_titanic(acquire.get_titanic_data(titanic_query, './'))

Using the Telco dataset

Use the function defined in acquire.py to load the Telco data.

In [79]:
telco_query = """
        SELECT * FROM customers
        JOIN contract_types USING (contract_type_id)
        JOIN internet_service_types USING (internet_service_type_id)
        JOIN payment_types USING (payment_type_id)
        """

In [80]:
telco = acquire.get_telco_data(telco_query, './')

In [78]:
# telco.info()

In [81]:
# check for null values:
telco.isna().sum()[telco.isna().sum() > 0]

Series([], dtype: int64)

In [60]:
pd.crosstab(telco.internet_service_type_id, telco.internet_service_type)

internet_service_type,DSL,Fiber optic,None
internet_service_type_id,,,
1,2421,0,0
2,0,3096,0
3,0,0,1526


In [61]:
telco.internet_service_type_id.unique()

array([1, 2, 3])

In [ ]:
# pd.Series.value_counts?

In [62]:
telco.internet_service_type.value_counts(dropna=False)

Fiber optic    3096
DSL            2421
None           1526
Name: internet_service_type, dtype: int64

In [63]:
telco.internet_service_type

0               DSL
1               DSL
2       Fiber optic
3       Fiber optic
4       Fiber optic
           ...     
7038            DSL
7039    Fiber optic
7040            DSL
7041            DSL
7042            DSL
Name: internet_service_type, Length: 7043, dtype: object

In [64]:
[print(col) for col in telco.columns]

Unnamed: 0
payment_type_id
internet_service_type_id
contract_type_id
customer_id
gender
senior_citizen
partner
dependents
tenure
phone_service
multiple_lines
online_security
online_backup
device_protection
tech_support
streaming_tv
streaming_movies
paperless_billing
monthly_charges
total_charges
churn
contract_type
internet_service_type
payment_type


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Drop any unnecessary, unhelpful, or duplicated columns. This could mean dropping foreign key columns but keeping the corresponding string values, for example.

Handle null values.

Create a function named prep_telco that accepts the raw telco data, and returns the data with the transformations above applied.

In [66]:
[col for col in telco.columns if col.endswith('_id')]

['payment_type_id',
 'internet_service_type_id',
 'contract_type_id',
 'customer_id']

In [67]:
# sanity check for 1:1 ratio on ids:
pd.crosstab(telco.contract_type_id, telco.contract_type)

contract_type,Month-to-month,One year,Two year
contract_type_id,,,
1,3875,0,0
2,0,1473,0
3,0,0,1695


In [68]:
pd.crosstab(telco.payment_type_id, telco.payment_type)

payment_type,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
payment_type_id,,,,
1,0,0,2365,0
2,0,0,0,1612
3,1544,0,0,0
4,0,1522,0,0


In [69]:
def prep_telco(telco) -> pd.DataFrame:
    '''
    prep_telco will take in a a single pandas dataframe
    presumed of the same structure as presented from 
    the acquire module's get_telco_data function (refer to acquire docs)
    returns a single pandas dataframe with redudant columns
    removed and missing values filled.
    '''
    telco = telco.drop(
    columns=[
        'Unnamed: 0',
        'internet_service_type_id',
        'payment_type_id',
        'contract_type_id',   
    ])
    telco.loc[:,'internet_service_type'] = telco.internet_service_type.\
    fillna('no internet')
    telco = telco.set_index('customer_id')
    telco.loc[:,'total_charges'] = (telco.total_charges + '0')
    telco.total_charges = telco.total_charges.astype(float)
    return telco

In [70]:
telco.total_charges.astype('float')

ValueError: could not convert string to float: ' '